In [4]:
from pyxnat import Interface
config_file = '/home/grg/.xnat_ci.cfg'
central = Interface(config=config_file, verify=False)
projects = central.select.projects().get()

sizes = {}
for project in projects:
    
    p = central.select.project(project).subjects().get()
    e = central.select.project(project).experiments().get()
    sizes[project] = (len(p), len(e))
    
import operator 
sizes = sorted(sizes.items(), key=operator.itemgetter(1), reverse=True)
print sizes

import warnings
warnings.filterwarnings('ignore')

[('testenv', (4, 7))]


In [5]:
from pyxnat import Interface
config_file = '/home/grg/.xnat_ci.cfg'
central = Interface(config=config_file, verify=False)
projects = central.select.projects().get()

project = 'testenv'

# Collecting data from XNAT
subjects = central.select.project(project).subjects()
info = []
for s in subjects:
    try:
        sid = s.get().split('xnat:dcmPatientId>')[1].split('</')[0]
        sname = s.get().split('xnat:dcmPatientName>')[1].split('</')[0]
        birthdate = s.get().split('xnat:dcmPatientBirthDate>')[1].split('</')[0]
        scandate = s.get().split('xnat:date>')[1].split('</')[0]
        info.append([sid, sname, birthdate, scandate])
    except IndexError:
        print s.label(), s.id(), 'error'
  
# Building DataFrame
import pandas as pd
df = pd.DataFrame(info, columns=('PatientId', 'PatientName', 'Birthdate', 'Scandate'))
df = pd.DataFrame(info, index=df['PatientId'], columns=('PatientId', 'PatientName', 'Birthdate', 'Scandate'))
df = df.sort_index()
del df['PatientId']
df.head()

# Saving it in an Excel table
ew = pd.ExcelWriter('/tmp/%s.xls'%project, encoding='utf-8')
df.to_excel(ew)
ew.save()